In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

from keras import layers
from keras import regularizers

import re

In [2]:
from keras import backend as K
K.clear_session()

In [3]:
physical_devices = tf.config.list_physical_devices()
CLASSES = 48
IMAGE_SIZE = 24

print(physical_devices)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
tf.config.set_visible_devices(physical_devices[1], 'GPU')

## Building Dataset

### Attributes description

- a1 - face containing flag: (1-with face, 0-without face),
- a2 - image number in current class (person) beginning from 0,
- a3 - class (person) number beginning from 0,
- a4 - sex (0 - woman, 1 - man)
- a5 - race (0- white, 1 - negro, 2 - indian, ...)
- a6 - age (0 - baby, 1 - young, 2 - middle-age, 3 - old) 
- a7 - binokulars (0 - without, 1 - transparent, 2 - dark)
- a8 - emotional expression (not state!) (0 - sad, 1 - neutral, 2 - happy)

In [5]:
def read_file(filename):
  
  with open(filename, 'r') as file:
    lines = file.readlines()
    
    # Remove newLines
    for i, line in enumerate(lines):
      lines[i] = line.replace('\n', '')
    
    # We assume these are integers
    EXAMPLES_NR = int(lines[0])
    PIXELS_NR = int(lines[1])
    
    # inputs = np.zeros(EXAMPLES_NR)
    inputs = list()
    labels = np.zeros(EXAMPLES_NR, dtype=int)
    
    examples_raw = lines[2:EXAMPLES_NR+2]
    
    for i, example_raw in enumerate(examples_raw):
      # Split by spaces (treats multiple as one)
      tokens = re.split('\s+', example_raw)
      
      pixel_values = np.array(tokens[0:PIXELS_NR])
      attributes = tokens[PIXELS_NR:]
      
      pixel_values = np.array(pixel_values, dtype=float)
      pixel_values = pixel_values.reshape([IMAGE_SIZE, IMAGE_SIZE])
      
      inputs.append(pixel_values)
      labels[i] = int(attributes[2])
      
    inputs = np.array(inputs)
  return inputs, labels

In [6]:
# classes 0-15
X_0, y_0 = read_file('./data/x24x24.txt')
# classes 16-31
X_1, y_1 = read_file('./data/y24x24.txt')
# # classes 32-48
X_2, y_2 = read_file('./data/z24x24.txt')

print(len(X_0), len(y_0))
print(len(X_1), len(y_1))
print(len(X_2), len(y_2))

2321 2321
2208 2208
2306 2306


In [7]:
from sklearn.model_selection import train_test_split

# Concatenate train and test images
X = np.concatenate((X_0, X_1, X_2))
y = np.concatenate((y_0, y_1, y_2))
print(len(X), len(y))

N_TRAIN_EXAMPLES=int(len(X) * 0.8)
N_TEST_EXAMPLES=len(X) - N_TRAIN_EXAMPLES


X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=N_TRAIN_EXAMPLES, test_size=N_TEST_EXAMPLES, random_state=1)

# Getting dummy variables
y_train_fixed = np.zeros((y_train.shape[0], CLASSES))
# y_valid_fixed = np.zeros((y_valid.shape[0], 10))
y_test_fixed = np.zeros((y_test.shape[0], CLASSES))

for i, value in enumerate(y_train):
  y_train_fixed[i][value] = 1
  
for i, value in enumerate(y_test):
  y_test_fixed[i][value] = 1

6835 6835


In [8]:
print(len(X_test))

1367


## Building a model

AlexNet implementation in keras:

```py
  keras.Input(shape=(227, 227, 3)),
  layers.Conv2D(96, kernel_size=11, strides=4, padding='same', activation="relu"),
  layers.MaxPooling2D(pool_size=3, strides=2),
  layers.Conv2D(256, kernel_size=5, padding='same', activation="relu"),
  layers.MaxPooling2D(pool_size=3, strides=2),
  layers.Conv2D(384, kernel_size=3, padding='same', activation="relu"),
  layers.Conv2D(384, kernel_size=3, padding='same', activation="relu"),
  layers.Conv2D(256, kernel_size=3, padding='same', activation="relu"),
  layers.MaxPooling2D(pool_size=3, strides=2),
  layers.Dropout(0.5),
  layers.Dense(4096, activation="relu"),
  layers.Dropout(0.5),
  layers.Dense(4096, activation="relu"),
  layers.Dense(CLASSES, activation="softmax")
```

In [21]:
conv_regularizer = regularizers.l2(l=0.01)
dense_regularizer = regularizers.l2(l=0.01)

# model = keras.Sequential(
#   [
#     keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 1)),
#     layers.Conv2D(80, kernel_size=3, strides=1, padding='same', activation="relu", kernel_regularizer=conv_regularizer),
#     layers.MaxPooling2D(pool_size=2, strides=1),
#     layers.Conv2D(112, kernel_size=5, padding='same', activation="relu", kernel_regularizer=conv_regularizer),
#     layers.MaxPooling2D(pool_size=3, strides=2),
#     layers.Conv2D(160, kernel_size=3, padding='same', activation="relu", kernel_regularizer=conv_regularizer),
#     layers.Conv2D(160, kernel_size=3, padding='same', activation="relu", kernel_regularizer=conv_regularizer),
#     layers.Conv2D(112, kernel_size=3, padding='same', activation="relu", kernel_regularizer=conv_regularizer),
#     layers.MaxPooling2D(pool_size=3, strides=2),
#     layers.Flatten(),
#     # layers.Dropout(0.4),
#     layers.Dense(768, activation="relu", kernel_regularizer=dense_regularizer),
#     # layers.Dropout(0.4),
#     layers.Dense(768, activation="relu", kernel_regularizer=dense_regularizer),
#     layers.Dense(CLASSES, activation="softmax"),
    
#   ]
# )

model = keras.Sequential(
  [
    keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 1)),
    layers.Conv2D(80, kernel_size=3, strides=1, padding='same', activation="relu"),
    layers.MaxPooling2D(pool_size=2, strides=1),
    layers.Conv2D(112, kernel_size=5, padding='same', activation="relu"),
    layers.MaxPooling2D(pool_size=3, strides=2),
    layers.Conv2D(160, kernel_size=3, padding='same', activation="relu"),
    layers.Conv2D(160, kernel_size=3, padding='same', activation="relu"),
    layers.Conv2D(112, kernel_size=3, padding='same', activation="relu"),
    layers.MaxPooling2D(pool_size=3, strides=2),
    layers.Flatten(),
    # layers.Dropout(0.4),
    layers.Dense(768, activation="relu"),
    # layers.Dropout(0.4),
    layers.Dense(768, activation="relu"),
    layers.Dense(CLASSES, activation="softmax"),
    
  ]
)

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 24, 24, 80)        800       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 23, 23, 80)       0         
 g2D)                                                            
                                                                 
 conv2d_21 (Conv2D)          (None, 23, 23, 112)       224112    
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 11, 11, 112)      0         
 g2D)                                                            
                                                                 
 conv2d_22 (Conv2D)          (None, 11, 11, 160)       161440    
                                                                 
 conv2d_23 (Conv2D)          (None, 11, 11, 160)      

In [22]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

## Training Preparation

In [23]:
CHECKPOINT_PATH = 'models/checkpoints/famous48_test_model_v1_best.keras'

In [24]:
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
  filepath=CHECKPOINT_PATH,
  save_weights_only=False,
  monitor='val_accuracy',
  mode='max',
  save_best_only=True,  
)

In [25]:
%%time
batch_size = 100
epochs = 250

history = model.fit(
  X_train, y_train_fixed,
  batch_size=batch_size,
  epochs=epochs,
  validation_split=0.2,
  shuffle=True,
  callbacks=[model_checkpoint_callback]
)

Epoch 1/250


44/44 [==============================] - 2s 32ms/step - loss: 3.8234 - accuracy: 0.0313 - val_loss: 3.7135 - val_accuracy: 0.0530
Epoch 2/250
44/44 [==============================] - 1s 28ms/step - loss: 3.3728 - accuracy: 0.1072 - val_loss: 3.0271 - val_accuracy: 0.1417
Epoch 3/250
44/44 [==============================] - 1s 28ms/step - loss: 2.7000 - accuracy: 0.2293 - val_loss: 2.5715 - val_accuracy: 0.2614
Epoch 4/250
44/44 [==============================] - 1s 28ms/step - loss: 2.0702 - accuracy: 0.3980 - val_loss: 1.9256 - val_accuracy: 0.4141
Epoch 5/250
44/44 [==============================] - 1s 28ms/step - loss: 1.5395 - accuracy: 0.5281 - val_loss: 1.5648 - val_accuracy: 0.5256
Epoch 6/250
44/44 [==============================] - 1s 28ms/step - loss: 1.1825 - accuracy: 0.6280 - val_loss: 1.2644 - val_accuracy: 0.6069
Epoch 7/250
44/44 [==============================] - 1s 28ms/step - loss: 0.8363 - accuracy: 0.7318 - val_loss: 1.2556 - val_accuracy: 0.6335
Epoch 8/250
44/44 

KeyboardInterrupt: 